전처리 : Augmentation외 다른 전처리는 하지 않았습니다.

학습 :

1) EfficientNetB8 (imagenet + adversarial training pre-trained weight 사용)  
2) 5fold  
3) Augmentation : 90~360 회전, Cutout + Non-rigid transform 몇 개  
4) Adam, MultiLabelSoftMarginLoss, ReduceLROnPlateau Scheduler, Dropout=50  

후처리 : TTA (90~360회전), soft voting, prediction score > 0.35 이면 해당 class를 1로 라벨링

LB : 리더보드 TTA: Test Time Augmentation
단일모델 -> LB 0.8774 (thres 0.5)  
단일모델 + TTA -> LB 0.8841 (thres 0.5)  
5fold(hard vote) + TTA -> LB 0.8889 (thres 0.5)  
5fold(soft vote) + TTA -> LB 0.8901 (thres 0.5)  
5fold(soft vote) + TTA -> LB 0.8902 (thres 0.35) 입니다.  

여러가지 테스트 한 결과 참고 바랍니다.

1) Augmetnation : 코드에 있는 옵션 외 다른 옵션을 더 추가할 때 마다 validation loss는 줄지만, test score는 더 안 좋아졌습니다.  
2) 이미지 전처리 : 선형, 저주파 통과, bilateral 필터 등등 이것저것 해봤는데 성능이 더 안 좋아졌습니다. 눈으로 보기에는 bilateral 필터가 가장 효과가 좋았습니다.  
3) 이전 대회 데이터 : 이미지넷 + 이전 대회 이미지 학습 (크기도, 위치도 랜덤) 후, 대회 데이터 학습 하였는데 성능이 더 안 좋아졌습니다.  
4) gradient clipping : 코드에는 들어가 있지만, 큰 효과는 없는 것 같습니다.  
5) TTA : 회전 외 다른 옵션을 추가할 때마다 성능이 낮아졌습니다.  

Efficient Net 이란?
====

Neural Architecture Search(강화학습 기반의 network 탐색 기법)

1. Depth (d) (layer를 깊게 가져가는것, ex. ResNet-100 -> ResNet-1000)  

2. Width (w) (# of channels를 많게 가져가는것)  

3. Resolution (r) (input image의 사이즈를 MxM -> r*Mxr*M의, 더 큰사이즈로 입력받는것) 

=>위 세가지 요소를 Compund Scaling 하여 적절한 파라미터 설정

![파라미터 대비 성능](Efficient1.png)

![피쳐 추출 결과 예시](Parameter.png)

In [4]:
import os
import easydict
import pandas as pd
from pathlib import Path
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import random
import tqdm
import cv2
from efficientnet_pytorch import EfficientNet
import torch
import albumentations
import albumentations.pytorch

**Albumentaion Document**  
https://albumentations.ai/docs/  

**Albumentation GitHub**  
https://github.com/albumentations-team/albumentations  


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

args = easydict.EasyDict({
    "image_path": "./data/dirty_mnist_2nd/",
    "label_path": "./data/dirty_mnist_2nd_answer.csv",
    "kfold_idx": 0, ## 0~4까지 바꿔 가면서 5번 학습합니다. 

    "model": "efficientnet-b8",
    "epochs": 2000,
    "batch_size": 16,
    "lr": 1e-3,
    "patience": 8,

    "resume": None,
    "device": device,
    "comments": None,
})

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class DatasetMNIST(torch.utils.data.Dataset):
    def __init__(self, image_folder, label_df, transforms):        
        self.image_folder = image_folder   
        self.label_df = label_df
        self.transforms = transforms

    def __len__(self):
        return len(self.label_df)
    
    def __getitem__(self, index):        
        image_fn = self.image_folder +\
            str(self.label_df.iloc[index,0]).zfill(5) + '.png'
                                              
        image = cv2.imread(image_fn, cv2.IMREAD_GRAYSCALE)        
        image = image.reshape([256, 256, 1])

        label = self.label_df.iloc[index,1:].values.astype('float')

        if self.transforms:            
            image = self.transforms(image=image)['image'] / 255.0 #스케일링

        return image, label

mnist_transforms = {
    'train' : albumentations.Compose([ 
            albumentations.RandomRotate90(),
            albumentations.OneOf([ #다음 augmentation 
                albumentations.GridDistortion(distort_limit=(-0.3, 0.3), border_mode=cv2.BORDER_CONSTANT, p=1), #그리드 왜곡
                albumentations.ShiftScaleRotate(rotate_limit=15, border_mode=cv2.BORDER_CONSTANT, p=1), #Scale 및 Rotate       
                albumentations.ElasticTransform(alpha_affine=10, border_mode=cv2.BORDER_CONSTANT, p=1), #그리드 왜곡
            ], p=1),    
            albumentations.Cutout(num_holes=16, max_h_size=15, max_w_size=15, fill_value=0), #성겅성겅
            albumentations.pytorch.ToTensorV2(),
        ]),
    'valid' : albumentations.Compose([        
        albumentations.pytorch.ToTensorV2(),
        ]),
    'test' : albumentations.Compose([        
        albumentations.pytorch.ToTensorV2(),
        ]),
}

def train(train_loader, model, loss_func, device, optimizer, scheduler=None):
    n = 0
    running_loss = 0.0
    running_corrects = 0

    epoch_loss = 0.0
    epoch_acc = 0.0

    model.train()    

    with tqdm.tqdm(train_loader, total=len(train_loader), desc="Train", file=sys.stdout) as iterator:
        for train_x, train_y in iterator:
            train_x = train_x.float().to(device)
            train_y = train_y.float().to(device)
            
            output = model(train_x)
            
            loss = loss_func(output, train_y)
            
            n += train_x.size(0)
            running_loss += loss.item() * train_x.size(0)

            epoch_loss = running_loss / float(n)

            output = output > 0.5
            running_corrects += (output == train_y).sum()
            epoch_acc = running_corrects / train_y.size(1) / n

            log = 'loss - {:.5f}, acc - {:.5f}'.format(epoch_loss, epoch_acc)
            
            iterator.set_postfix_str(log)

            optimizer.zero_grad() #역전파 실행전 grad 0으로 초기화역전파 실행전 grad 0으로 초기화
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1) #gradient clipping
            optimizer.step()

    if scheduler:
        scheduler.step(epoch_loss)

    return epoch_loss, epoch_acc


def validate(valid_loader, model, loss_func, device, scheduler=None):
    n = 0
    running_loss = 0.0
    running_corrects = 0

    epoch_loss = 0.0
    epoch_acc = 0.0

    model.eval()

    with tqdm.tqdm(valid_loader, total=len(valid_loader), desc="Valid", file=sys.stdout) as iterator: #tqdm 상태바
        for train_x, train_y in iterator:
            train_x = train_x.float().to(device)
            train_y = train_y.float().to(device)

            with torch.no_grad():
                output = model(train_x)
            
            loss = loss_func(output, train_y)

            n += train_x.size(0)
            running_loss += loss.item() * train_x.size(0)

            epoch_loss = running_loss / float(n)

            output = output > 0.5
            running_corrects += (output == train_y).sum()
            epoch_acc = running_corrects / train_y.size(1) / n

            log = 'loss - {:.5f}, acc - {:.5f}'.format(epoch_loss, epoch_acc)

            iterator.set_postfix_str(log)

    if scheduler:
        scheduler.step(epoch_loss)

    return epoch_loss, epoch_acc

In [ ]:
print('=' * 50)
print('[info msg] arguments\n')
for key, value in vars(args).items():
    print(key, ":", value)
print('=' * 50)

assert os.path.isdir(args.image_path), 'wrong path'
assert os.path.isfile(args.label_path), 'wrong path'
if (args.resume):
    assert os.path.isfile(args.resume), 'wrong path'
assert args.kfold_idx < 5

seed_everything(777)

data_set = pd.read_csv(args.label_path)
valid_idx_nb = int(len(data_set) * (1 / 5))
valid_idx = np.arange(valid_idx_nb * args.kfold_idx, valid_idx_nb * (args.kfold_idx + 1))

print('[info msg] validation fold idx !!\n')        
print(valid_idx)
print('=' * 50)

train_data = data_set.drop(valid_idx)
valid_data = data_set.iloc[valid_idx]

train_set = DatasetMNIST(
    image_folder=args.image_path,
    label_df=train_data,
    transforms=mnist_transforms['train']
)

valid_set = DatasetMNIST(
    image_folder=args.image_path,
    label_df=valid_data,
    transforms=mnist_transforms['valid']
)

train_data_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=args.batch_size,
        shuffle=True,
    )

valid_data_loader = torch.utils.data.DataLoader(
        valid_set,
        batch_size=args.batch_size,
        shuffle=False,
    )

model = None

if(args.resume):
    model = EfficientNet.from_name(args.model, in_channels=1, num_classes=26, dropout_rate=0.5)
    model.load_state_dict(torch.load(args.resume))
    print('[info msg] pre-trained weight is loaded !!\n')        
    print(args.resume)
    print('=' * 50)

else:
    print('[info msg] {} model is created\n'.format(args.model))
    model = EfficientNet.from_pretrained(args.model, in_channels=1, num_classes=26, dropout_rate=0.5, advprop=True)
    print('=' * 50)

if args.device == 'cuda' and torch.cuda.device_count() > 1 :
    model = torch.nn.DataParallel(model)

model.to(args.device)

optimizer = torch.optim.Adam(model.parameters(), args.lr)
criterion = torch.nn.MultiLabelSoftMarginLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer,
    mode='min',
    patience=2,
    factor=0.5,
    verbose=True
    )

train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

best_loss = float("inf")

patience = 0

date_time = datetime.now().strftime("%m%d%H%M%S")
SAVE_DIR = os.path.join('./save', date_time)

print('[info msg] training start !!\n')
startTime = datetime.now()
for epoch in range(args.epochs):        
    print('Epoch {}/{}'.format(epoch+1, args.epochs))
    train_epoch_loss, train_epoch_acc = train(
        train_loader=train_data_loader,
        model=model,
        loss_func=criterion,
        device=args.device,
        optimizer=optimizer,
        )
    train_loss.append(train_epoch_loss)
    train_acc.append(train_epoch_acc)

    valid_epoch_loss, valid_epoch_acc = validate(
        valid_loader=valid_data_loader,
        model=model,
        loss_func=criterion,
        device=args.device,
        scheduler=scheduler,
        )
    valid_loss.append(valid_epoch_loss)        
    valid_acc.append(valid_epoch_acc)

    if best_loss > valid_epoch_loss:
        patience = 0
        best_loss = valid_epoch_loss

        Path(SAVE_DIR).mkdir(parents=True, exist_ok=True)
        torch.save(model.state_dict(), os.path.join(SAVE_DIR, 'model_best.pth.tar'))
        print('MODEL IS SAVED TO {}!!!'.format(date_time))
        
    else:
        patience += 1
        if patience > args.patience - 1:
            print('=======' * 10)
            print("[Info message] Early stopper is activated")
            break

elapsed_time = datetime.now() - startTime

train_loss = np.array(train_loss)
train_acc = np.array(train_acc)
valid_loss = np.array(valid_loss)
valid_acc = np.array(valid_acc)

best_loss_pos = np.argmin(valid_loss)

print('=' * 50)
print('[info msg] training is done\n')
print("Time taken: {}".format(elapsed_time))
print("best loss is {} w/ acc {} at epoch : {}".format(best_loss, valid_acc[best_loss_pos], best_loss_pos))    

print('=' * 50)
print('[info msg] {} model weight and log is save to {}\n'.format(args.model, SAVE_DIR))

with open(os.path.join(SAVE_DIR, 'log.txt'), 'w') as f:
    for key, value in vars(args).items():
        f.write('{} : {}\n'.format(key, value))            

    f.write('\n')
    f.write('total ecpochs : {}\n'.format(str(train_loss.shape[0])))
    f.write('time taken : {}\n'.format(str(elapsed_time)))
    f.write('best_train_loss {} w/ acc {} at epoch : {}\n'.format(np.min(train_loss), train_acc[np.argmin(train_loss)], np.argmin(train_loss)))
    f.write('best_valid_loss {} w/ acc {} at epoch : {}\n'.format(np.min(valid_loss), valid_acc[np.argmin(valid_loss)], np.argmin(valid_loss)))

plt.figure(figsize=(15,5))
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='train loss')
plt.plot(valid_loss, 'o', label='valid loss')
plt.axvline(x=best_loss_pos, color='r', linestyle='--', linewidth=1.5)
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(train_acc, label='train acc')
plt.plot(valid_acc, 'o', label='valid acc')
plt.axvline(x=best_loss_pos, color='r', linestyle='--', linewidth=1.5)
plt.legend()
plt.savefig(os.path.join(SAVE_DIR, 'history.png'))